In [12]:

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [13]:
# df = pd.read_csv('fixedTesting/predictionsBoth.csv')
df = pd.read_csv('predictionsNew.csv')
# df = df.drop(['taxa_obitos_covid',
#                 'taxa_atendimentos_dengue',
#                 'taxa_atendimentos_saude',
#                 'taxa_unidades_saude'], axis=1)
df


,Localidade,populacao_por_hectare,porcentagem_brancos,porcentagem_adultos,indiceEnvelhecimento,rendaMediacRendimento,rendaMedianacRendimento,porcentagem_cobertura,taxa_obitos_covid,taxa_atendimentos_covid,taxa_atendimentos_dengue,taxa_atendimentos_saude,taxa_unidades_saude,preds
0,ABRANCHES,30.56,0.774433,0.710668,32.04,1709.450000,1000.000000,0.375862,0.004398,0.334445,0.003564,0.661688,0.000152,0.334445
1,ÁGUA VERDE,107.74,0.911522,0.757355,92.35,4253.570000,2500.000000,0.169136,0.004064,0.267963,0.002450,0.000000,0.000194,0.267963
2,AHÚ,62.03,0.916913,0.722666,92.58,2381.596622,1467.405405,0.199552,0.003824,0.286807,0.001912,0.000000,0.000087,0.286807
3,ALTO BOQUEIRÃO,44.77,0.762386,0.721507,25.17,1262.610000,1000.000000,0.301180,0.004528,0.331333,0.016079,0.000000,0.000075,0.331333
4,ALTO DA GLÓRIA,63.37,0.921593,0.750721,133.22,4101.110000,3000.000000,0.148813,0.007030,0.416907,0.004326,0.000000,0.000541,0.416907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,UBERABA,50.71,0.757550,0.712113,25.46,1623.510000,1000.000000,0.141340,0.004746,0.360248,0.005371,0.711696,0.000083,0.360248
71,UMBARÁ,8.34,0.763641,0.699092,16.35,1164.040000,850.000000,0.412739,0.004805,0.381367,0.003257,0.000000,0.000107,0.381367
72,VILA IZABEL,95.44,0.909130,0.761068,75.57,3390.900000,2181.500000,0.164478,0.005340,0.405857,0.003359,0.279328,0.000258,0.405857
73,VISTA ALEGRE,30.28,0.865167,0.709081,55.81,2898.740000,1500.000000,0.332934,0.004108,0.345031,0.003215,0.517546,0.000089,0.345031


In [14]:
import ipywidgets as widgets
from IPython.display import display


gdf = gpd.read_file('DIVISA_DE_BAIRROS/DIVISA_DE_BAIRROS.shp')
gdf = gdf.merge(df, left_on='NOME', right_on='Localidade', how='right')

gdf['error'] = gdf['taxa_atendimentos_covid'] - gdf['preds']
gdf['error_percentage'] = gdf['preds']/gdf['taxa_atendimentos_covid']


w = widgets.Dropdown(
    options=list(gdf),
    value='Localidade',
    description='Data:',
)

map_output = widgets.Output()

display(w, map_output)

Dropdown(description='Data:', index=10, options=('OBJECTID', 'CODIGO', 'TIPO', 'NOME', 'FONTE', 'CD_REGIONA', …

Output()

In [15]:
from IPython.display import clear_output

def update_map(change):
    with map_output:
        clear_output(wait=True)
        display(gdf.explore(column=change.new, legend=True, legend_kwds={"label": change.new, "orientation": "vertical"}))

w.observe(update_map, names='value')

# Initial map display
with map_output:
    display(gdf.explore(column=w.value, legend=True, legend_kwds={"label": w.value, "orientation": "vertical"}))

In [16]:
# Save the GeoDataFrame to a GeoJSON file
# gdf.to_file("curitiba_bairros_completo.geojson", driver='GeoJSON')


In [17]:
# # Create maps for each variable and save to file
# for column in gdf.columns:
#     if column != 'geometry':  # Skip the geometry column
#         fig, ax = plt.subplots(figsize=(12, 8))
#         gdf.plot(column=column, ax=ax, legend=True, cmap='viridis', edgecolor='grey', linewidth=0.2)
#         plt.title(f'Mapa de {column}')
#         plt.axis('off')
#         plt.tight_layout()
#         plt.savefig(f'maps/map_{column}.png', dpi=300, bbox_inches='tight')
#         plt.close(fig)  # Close the figure to free up memory

In [18]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


import numpy as np

# Define the custom color map
# cmap = mcolors.LinearSegmentedColormap.from_list("Diverging", ['bwr'])
divnorm=mcolors.TwoSlopeNorm(vmin=min(gdf['error']), vcenter=0., vmax=max(gdf['error']))
fig, ax = plt.subplots(figsize=(12, 8))
gdf.plot(column='error', ax=ax, legend=True, cmap='bwr', norm=divnorm, edgecolor='grey', linewidth=0.2)  # Added edgecolor and linewidth
plt.title(f'Mapa do erro absoluto')
plt.axis('off')
plt.tight_layout()
plt.savefig(f'map_error.png', dpi=300, bbox_inches='tight')
plt.close(fig)  # Close the figure to free up memory